In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn import tree
from sklearn import ensemble
from sklearn import datasets, linear_model
from sklearn import cross_validation, svm
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import scipy.io
train = scipy.io.loadmat('ReducedImagesForTraining.mat')['images'].T
test = scipy.io.loadmat('ReducedImagesForTesting.mat')['images'].T
print train.shape
from sklearn import (datasets, decomposition, ensemble, manifold, random_projection)


def pca(X, wsp):
    n,m = np.shape(X)
    X_mean = np.mean(X, axis = 0)
    X_std = np.std(X,axis = 0)
    X_standarized = (X - X_mean)/X_std
    S = np.cov(X_standarized.T)
    D,V = np.linalg.eigh(S)
    #rint "suma wartosci wlasnych:",np.sum(D)
    Y = np.dot(V.T,X_standarized.T).T
    D_bis = np.matrix(1/np.sqrt(D)).reshape(m,1)
    Y_new = np.multiply(D_bis,Y.T).T
    k = 1; 
    suma = np.sum(D)
    while(np.sum(D[-k:])/suma < wsp):
        k = k+1
    return Y_new[:,-k:], D[-k:], V[:,-k:], k


def PCA(X, k):
    n,m = np.shape(X)
    X_mean = np.mean(X, axis = 0)
    X_std = np.std(X,axis = 0)
    X_standarized = (X - X_mean)/X_std
    S = np.cov(X_standarized.T)
    D,V = np.linalg.eigh(S)
    #print "suma wartosci wlasnych:",np.sum(D)
    Y = np.dot(V.T,X_standarized.T).T
    D_bis = np.matrix(1/np.sqrt(D)).reshape(m,1)
    Y_new = np.multiply(D_bis,Y.T).T
    #print Y_new
    return Y_new[:,-k:]


def nearestPoints(knownData,testData,k):
    D1 = np.sum(np.square(knownData),axis = 1,keepdims=True)
    D2 = np.sum(np.square(testData),axis = 1,keepdims=True).T
    M = D1 + D2 - 2. * np.dot(knownData,testData.T)
    #print M
    return np.asarray(np.argsort(M,axis=0)[:k,:].T)
    

def KNN(knownData, testData, knownClass, testClass,k):
    error = 0
    nearest = nearestPoints(knownData,testData,k)
    #print nearest
    length = testData.shape[0]
    nearestClass = np.empty(length)
    error_list = []
    for i in range(0,length):
        #print i,nearest[i]
        nearestClass[i] = np.argmax(np.bincount([knownClass[a] for a in nearest[i]]))
        if(not(nearestClass[i] == testClass[i])):
           error += 1
           error_list = np.append(error_list, [i])
    return error, error_list

from scipy import misc
from IPython.display import Image
from IPython.display import display
def pokaz(i):
    zdj1 = test[i,:]
    plt.figure(figsize=(10,4))
    plt.imshow(zdj1.reshape(60,82).T, cmap = 'gray')
knownClass = [int((k-1)/5) + 1 for k in range(1,251)]
testClass = [int((k-1)/2) + 1 for k in range(1,101)]
#print knownClass
#print testClass

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


(250, 4920)


In [2]:
def crossValidation(data,dataClass,k):
    size = data.shape[0]
    arr = np.arange(size)
    np.random.shuffle(arr)
    err = 0
    n = int(size/10)
    for i in range(10):
        mask2 = arr[np.arange(i*n,min((i+1)*n,size))]
        mask1 = arr[np.concatenate((np.arange(i*n),np.arange((i+1)*n,size)))]
        er = KNN(data[mask1],data[mask2],dataClass[mask1],dataClass[mask2],k)
        #print er
        err += er[0]
    return float(err)/(size) * 100

In [32]:
def everything(X,Xclass,name):
    x1,x2 = zip(*np.random.permutation(zip(X,Xclass)))
    X1, X1class = np.asarray(x1),np.asarray(x2)
    suma = 0
    for i in range(1,6,2):
        suma += crossValidation(X,Xclass,i)
    print "Dla metody", name,float(suma)/3,"%"

In [34]:
X = np.concatenate((train,test))
y = np.concatenate((knownClass,testClass))
everything(X,y,"nieprzerobione dane")
for i in range(5,201,20):
    #everything(X,y, "nieprzerobionych danych")
    print "Odpalamy metody dla n_components=",i
    X_projected = random_projection.SparseRandomProjection(n_components=i).fit_transform(X)
    everything(X_projected,y, "Sparse Random Projection")
    X_pca = decomposition.PCA(n_components=i, whiten=True).fit_transform(X)
    everything(X_pca,y, "PCA")
    #X_kpca = decomposition.KernelPCA(n_components=i, kernel="rbf", gamma=10).fit_transform(X)
    #everything(X_kpca,y, "KernelPCA")
    X_iso = manifold.Isomap(30, n_components=i).fit_transform(X)
    everything(X_iso,y,"Isomap")
    X_lle = manifold.LocallyLinearEmbedding(30, n_components=i).fit_transform(X)
    everything(X_lle,y, "Locally Linear Embedding")
    X_mds = manifold.MDS(n_components=i, n_init=1, max_iter=100).fit_transform(X)
    everything(X_mds,y, "manifold MDE")
    #X_tsne = manifold.TSNE(n_components=i, init='pca').fit_transform(X) 
    #everything(X_tsne,y, "TSNE")
    

Dla metody nieprzerobione dane 56.6666666667 %
Odpalamy metody dla n_components= 5
Dla metody Sparse Random Projection 84.7619047619 %
Dla metody PCA 73.5238095238 %
Dla metody Isomap 77.7142857143 %
Dla metody Locally Linear Embedding 87.3333333333 %
Dla metody manifold MDE 85.0476190476 %
Odpalamy metody dla n_components= 25
Dla metody Sparse Random Projection 65.1428571429 %
Dla metody PCA 21.1428571429 %
Dla metody Isomap 69.7142857143 %
Dla metody Locally Linear Embedding 52.9523809524 %
Dla metody manifold MDE 78.0952380952 %
Odpalamy metody dla n_components= 45
Dla metody Sparse Random Projection 62.1904761905 %
Dla metody PCA 12.6666666667 %
Dla metody Isomap 68.1904761905 %
Dla metody Locally Linear Embedding 47.9047619048 %
Dla metody manifold MDE 76.2857142857 %
Odpalamy metody dla n_components= 65
Dla metody Sparse Random Projection 59.3333333333 %
Dla metody PCA 12.380952381 %
Dla metody Isomap 68.4761904762 %
Dla metody Locally Linear Embedding 45.7142857143 %
Dla metody 

In [36]:
for i in range(200,1001,50):
    #everything(X,y, "nieprzerobionych danych")
    print "Odpalamy metody dla n_components=",i
    X_projected = random_projection.SparseRandomProjection(n_components=i).fit_transform(X)
    everything(X_projected,y, "Sparse Random Projection")
    X_pca = decomposition.PCA(n_components=i).fit_transform(X)
    everything(X_pca,y, "PCA")
    #X_kpca = decomposition.KernelPCA(n_components=i, kernel="rbf", gamma=10).fit_transform(X)
    #everything(X_kpca,y, "KernelPCA")
    X_iso = manifold.Isomap(30, n_components=i).fit_transform(X)
    everything(X_iso,y,"Isomap")
    #X_lle = manifold.LocallyLinearEmbedding(30, n_components=i).fit_transform(X)
    #everything(X_lle,y, "Locally Linear Embedding")
    X_mds = manifold.MDS(n_components=i, n_init=1, max_iter=100).fit_transform(X)
    everything(X_mds,y, "manifold MDE")
    #X_tsne = manifold.TSNE(n_components=i, init='pca').fit_transform(X) 
    #everything(X_tsne,y, "TSNE")

Odpalamy metody dla n_components= 200
Dla metody Sparse Random Projection 59.1428571429 %
Dla metody PCA 56.380952381 %
Dla metody Isomap 98.380952381 %
Dla metody manifold MDE 69.4285714286 %
Odpalamy metody dla n_components= 250
Dla metody Sparse Random Projection 60.5714285714 %
Dla metody PCA 55.8095238095 %
Dla metody Isomap 98.380952381 %
Dla metody manifold MDE 70.0 %
Odpalamy metody dla n_components= 300
Dla metody Sparse Random Projection 58.7619047619 %
Dla metody PCA 57.1428571429 %
Dla metody Isomap 98.4761904762 %
Dla metody manifold MDE 70.9523809524 %
Odpalamy metody dla n_components= 350
Dla metody Sparse Random Projection 58.2857142857 %
Dla metody PCA 57.0476190476 %
Dla metody Isomap 97.9047619048 %
Dla metody manifold MDE 69.8095238095 %
Odpalamy metody dla n_components= 400
Dla metody Sparse Random Projection 57.0476190476 %
Dla metody PCA 56.8571428571 %
Dla metody Isomap 98.4761904762 %
Dla metody manifold MDE 70.4761904762 %
Odpalamy metody dla n_components= 450

In [38]:
for i in range(10,101,5):
    print i
    X_pca = decomposition.PCA(n_components=i, whiten=True).fit_transform(X)
    everything(X_pca,y, "PCA")

10
Dla metody PCA 51.9047619048 %
15
Dla metody PCA 36.0 %
20
Dla metody PCA 29.8095238095 %
25
Dla metody PCA 21.4285714286 %
30
Dla metody PCA 17.2380952381 %
35
Dla metody PCA 13.5238095238 %
40
Dla metody PCA 12.0952380952 %
45
Dla metody PCA 11.8095238095 %
50
Dla metody PCA 12.1904761905 %
55
Dla metody PCA 13.3333333333 %
60
Dla metody PCA 11.619047619 %
65
Dla metody PCA 13.1428571429 %
70
Dla metody PCA 14.0952380952 %
75
Dla metody PCA 15.3333333333 %
80
Dla metody PCA 16.1904761905 %
85
Dla metody PCA 17.2380952381 %
90
Dla metody PCA 16.4761904762 %
95
Dla metody PCA 19.1428571429 %
100
Dla metody PCA 20.2857142857 %
